In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import scipy.stats
from IPython.display import Video, display, HTML
sns.set()

def read_csvx(path, *args, **kwargs):
    try:
        return pd.read_csv(str(path) +'.gz', *args, **kwargs)
    except Exception as e:
        return pd.read_csv(str(path), *args, **kwargs)

def followorbit(vals, path):
    out = []
    for n in range(100):
        closest05 = np.clip(round(path[n] * 2) / 2, 0, 10)
        rank = vals.query(f'nbopp == {n} and othercoop == {closest05}')['rank'].values[0]
        out.append(rank)
    return np.array(out)

import re
def gethighest(vals, prefix="logall_"):
        m = np.argmax([int(re.search(f'{prefix}(\d+)', val).group(1)) for val in vals])
        return vals[m]

    
def readparams(file):
    params = {} 
    with open(file) as f:
        for line in f:
            line = line[:-1]  # remove \n
            if not line or line.startswith('#') or line.startswith('import'):
                continue
            key, val = line.split('=')
            if val.lower() == 'true' or val.lower() == 'false':
                params[key] = val.lower() == 'true'
            else:
                try:
                    params[key] = float(val)
                except ValueError:
                    params[key] = val
    return params

In [ ]:
import platform
hostname = platform.node()

if hostname == 'cluster':
    prefix = '/home/ecoffet/robocoop/'
else:
    prefix = '/home/pecoffet/remoterobo/'

In [ ]:
dofolloworbit = True
for path in sorted(
            glob.glob(f"{prefix}/logs/lionscross2/*/*/*/*/*/rep00/", )
        ):
    
    display(HTML(f'<h3>{path}</h3>'))
    fig, axs_g = plt.subplots(2, 2, figsize=(10, 10))
    axs = axs_g.flatten()
    cura = 0
    try:
        fit = read_csvx(path + "fitnesslog.txt", delimiter="\t")
    except Exception as e:
        print(e)
    else:
        fit = fit.groupby(['gen', 'ind'], as_index=False).sum()
        sns.lineplot(y='fitness', x='gen', data=fit, ci=None, estimator='median', label="mediane", ax=axs[cura])
        axs[cura].legend()
        axs[cura].set_title('Fitness')
        #sns.lineplot(y='fitness', x='gen', data=fit, ci=None, estimator='min')
    
    
    try:
        logall = read_csvx(gethighest(glob.glob(path + 'logall*')), delimiter="\t")
    except Exception as e:
        plt.show()
        print(e)
        continue
    
    params = readparams(glob.glob(path + 'properties*')[0])
    indcoop = params.get('independantCoop', 0)
    nbMode = logall['nbOnOpp'].mode()[0]
    nbOpti = params.get('nOpti', 2)
    logall['ess'] = params['meanA'] / logall['nbOnOpp']
    ess2 = params['meanA'] / 2
    so2 = params['meanA'] + params['b'] - (params['b'] /2)
    logall['so'] = params['meanA'] + params['b'] - (params['b'] /logall['nbOnOpp'])
    logall['oppId'] = logall['oppId'].astype('category')
    if params.get('additiveVar', False):
        logall['trueCoop'] = logall['curCoopNoCoef'] + logall['fakeCoef'] - 1
    else:
        logall['trueCoop'] = logall['curCoopNoCoef'] if indcoop > 0 else logall['curCoopNoCoef'] * logall['fakeCoef']
    
    # print if ind coop or not
    print('ind coop =', indcoop)
    
    cura +=1
    # Coop corr
    #logall[['previt', 'prevother', 'prevoppid']] = logall.shift(periods=logall['id'].max()+1)[['iter', 'otherCoop', 'oppId']]
    #sns.regplot('curCoopNoCoef', 'otherCoop', logall.query('nbOnOpp == 2 and prevother == otherCoop and oppId == prevoppid').groupby(['id']).mean())
    axs[cura].set_ylim(0, so2 + 0.5)
    axs[cura].set_xlim(0, so2 + 0.5)
    logall['normothercoop'] = logall['otherCoop'] / (logall['nbOnOpp'].clip(2, None) - 1)
    sns.regplot('trueCoop', 'normothercoop', logall.query(f'nbOnOpp == {nbOpti}').groupby(['id']).mean(), label="opti", ax=axs[cura])
    sns.regplot('trueCoop', 'normothercoop', logall.query(f'nbOnOpp == {nbMode}').groupby(['id']).mean(), label="mode", ax=axs[cura])
    axs[cura].axhline(ess2)
    axs[cura].axhline(so2)
    axs[cura].legend()
    axs[cura].set_title('Satisficing')
    
    
    # Coop graph
    cura += 1
    coopval = logall.query(f'nbOnOpp == {nbMode} or nbOnOpp == {nbOpti}').groupby(['nbOnOpp', 'id'], as_index=False).mean()
    sns.violinplot(data=coopval, x='nbOnOpp', y='curCoopNoCoef', bw=0.02, ax=axs[cura])
    axs[cura].set_title('Distribution de la coopération')
    axs[cura].set_ylim(-0.1, params['meanA'] + params['b'] / nbMode + 1)
    order = sorted([nbMode, nbOpti])
    for i in range(1, 2):
        a = params['meanA'] / order[i-1]
        b = params['meanA'] + params['b'] * order[i-1] / (order[i-1]+1)
        axs[cura].plot([i-1.5, i-0.5], [b, b], c="b")
        axs[cura].plot([i-1.5, i-0.5], [a, a], c="r")

    
    # nbOnOpp distplot
    cura += 1
    axs[cura].set_title('Distribution du nombre d\'agent par opportunité')
    sns.distplot(logall['nbOnOpp'].dropna(), kde=False, ax=axs[cura], bins=range(0, 22))
    fig.tight_layout()
    plt.show(fig)
    if False:
        # Occupancy
        plt.figure(figsize=(20, 10))
        occupancy = logall.query('eval == 0 and 1000 < iter < 1250').groupby(['iter', 'oppId'], as_index=False).count()
        occupancy['logid'] = np.log(occupancy['id'])
        pivot = occupancy.pivot_table('id', 'oppId', 'iter', fill_value=0)
        try:
            sns.heatmap(pivot)
        except Exception:
            print(pivot)
        plt.show()
        allopps = logall['oppId'].unique()
        for ind in [0, 10, 20]:
            subdf = logall.query(f'id == {ind} and eval == 0 and 1000 < iter < 1250')
            a = subdf.pivot_table('id', 'iter', 'oppId', aggfunc='count', fill_value=0)
            # trick for add all the opps
            for oppval in allopps:
                if oppval not in a:
                    a[oppval] = 0
            a.sort_index(axis=1, inplace=True)
            a = a.transpose()
            # end of trick 

            fig, axs = plt.subplots(3, 1, figsize=(15, 12), sharex=True)

            sns.lineplot(y='nbOnOpp', x='iter', sort=False, data=subdf, ax=axs[0])

            sns.lineplot(y='ess', x='iter', sort=False, data=subdf, ax=axs[1], label="ess")
            sns.lineplot(y='so', x='iter', sort=False, data=subdf, ax=axs[1], label="so")
            sns.lineplot(y='curCoopNoCoef', x='iter', sort=False, data=subdf, ax=axs[1], label="played")
            sns.heatmap(a, ax=axs[2], cbar=False)
            axs[0].set_ylim(0, None)
            axs[1].set_ylim(0, 10)
            axs[1].legend()
            plt.show()
            


In [ ]:
%load_ext Cython

In [ ]:
rob = 0
print(df.loc[(df['id'] == rob) & (df['iter'] == i), 'otherCoop'])
i = 1
df = logall
df.loc[(df['id'] == rob) & (df['iter'] == i-1), 'prevother']= df.loc[(df['id'] == rob) & (df['iter'] == i), 'otherCoop']

In [ ]:
%%cython
def putprev(df):
    df['prevother'] = False
    for rob in df['id'].unique():
        for i in range(1, df['iter'].max()):
            df.loc[(df['id'] == rob) & (df['iter'] == i-1), 'prevother'] = df.loc[(df['id'] == rob) & (df['iter'] == i), 'otherCoop']

In [ ]:
logall[['previt', 'prevother', 'previd']] = logall.shift(periods=df['id'].max()+1)[['iter', 'otherCoop', 'id']]

In [ ]:
logall

In [ ]:
params